In [2]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string
import os
import re
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Youtube01-Psy.csv') #read the CSV file

In [4]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [5]:
column = ['COMMENT_ID','AUTHOR','DATE']
df.drop(columns = column,inplace=True)
df.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [6]:
df.CLASS.value_counts()

1    175
0    175
Name: CLASS, dtype: int64

In [8]:
df.shape

(350, 2)

In [10]:
df.drop_duplicates(inplace=True)
df.shape

(349, 2)

In [11]:
df.isnull().sum()

CONTENT    0
CLASS      0
dtype: int64

In [12]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
sent1 = "Subject: why is this"
sent2 = re.sub("Subject:","",sent1)
tokens = word_tokenize(sent1.lower())


In [14]:
def clean_txt(sent):    
    #Stripping white spaces before and after the text
    sent = sent.strip()
    #Replacing multiple spaces with a single space
    result = re.sub("\s+", " ", sent)
    result = re.sub("Subject:","",result)
    #Replacing Non-Alpha-numeric and non space charecters with nothing
    result = re.sub("[^\w\s]","",result)
    
    tokens = word_tokenize(result.lower())
    stop_updated = stopwords.words("english")  +  ["would", "could","told"]
    text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [24]:
df['ctext'] = df.CONTENT.apply(clean_txt)

In [25]:
df.head()

,CONTENT,CLASS,ctext
0,"Huh, anyway check out this you[tube] channel: ...",1,huh anyway check youtube channel kobyoshi02
1,Hey guys check out my new channel and our firs...,1,hey guys check new channel first vid monkeys m...
2,just for test I have to say murdev.com,1,test say murdevcom
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,shaking sexy ass channel enjoy
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1,watchvvtarggvgtwq check


In [28]:
X_text = df.ctext.values
y = df.CLASS.values

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size = 0.20, random_state=311)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_trainmat = tfidf_vectorizer.fit_transform(X_train)

In [31]:
y.sum()

175

In [32]:
y_train.sum()

140

In [33]:
X_trainmat.toarray()

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0.5, 0.5, 0.5],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

USING SVM Linear kernel

In [34]:
from sklearn.svm import SVC
classifier = SVC(kernel= 'linear')
classifier.fit(X_trainmat.toarray(), y_train)

SVC(kernel='linear')

In [35]:
X_testmat = tfidf_vectorizer.transform(X_test)

In [36]:
y_pred = classifier.predict(X_testmat.toarray())

In [37]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [38]:
accuracy_score(y_test,y_pred)

0.8714285714285714

In [39]:
y_predtrain = classifier.predict(X_trainmat.toarray())

In [40]:
accuracy_score(y_train,y_predtrain)

1.0

In [41]:
confusion_matrix(y_train,y_predtrain)

array([[139,   0],
       [  0, 140]], dtype=int64)

In [42]:
confusion_matrix(y_test,y_pred)

array([[28,  7],
       [ 2, 33]], dtype=int64)

Using Logistic Regression

In [43]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size = 0.20, random_state=2505)
classifier = LogisticRegression()

In [44]:
classifier.fit(X_trainmat.toarray(), y_train)

LogisticRegression()

In [45]:
y_pred = classifier.predict(X_testmat)

In [46]:
accuracy_score(y_test,y_pred)

0.5

In [47]:
confusion_matrix(y_test,y_pred)

array([[15, 16],
       [19, 20]], dtype=int64)

In [48]:
y_predtrain = classifier.predict(X_trainmat)

In [49]:
accuracy_score(y_train,y_predtrain)

0.9390681003584229